# Hypothesis Testing

In this notebook we demonstrate formal hypothesis testing using the NHANES data.

It is important to note that the NHANES data are a "complex survey". The data are not an independent and representative sample from the target population. Proper analysis of complex survey data should make use of additional information about how the data were collected. Since complex survey analysis is a somewhat specialized topic, we ignore this aspect of the data here, and analyze the NHANES data as if it were an independent and identically distributed sample from a population.

First we import the libraries that we will need.

In [1]:
library(tidyverse)
library(BSDA)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: 'BSDA'


The following object is masked from 'package:datasets':

    Orange




Below we read the data, and convert some of the integer codes to text values. The NHANES codebooks for `SMQ020`, `RIAGENDR`, and `DMDCITZN` describe the meanings of the numerical codes.

In [2]:
da <- read.csv('nhanes_2015_2016.csv')
head(da)

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,⋯,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,83732,1,NA,1,1,1,62,3,1,5,⋯,124,64,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2
2,83733,1,NA,6,1,1,53,3,2,3,⋯,140,88,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NA
3,83734,1,NA,NA,1,1,78,3,1,3,⋯,132,44,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2
4,83735,2,1,1,2,2,56,3,1,5,⋯,134,68,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2
5,83736,2,1,1,2,2,42,4,1,4,⋯,114,54,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2
6,83737,2,2,NA,2,2,72,1,2,2,⋯,122,58,64.4,150.0,28.6,34.4,33.5,31.4,92.9,NA


In [3]:
unique(da$RIAGENDR)

[1] 1 2

In [4]:
table(da$SMQ020)


   1    2    7    9 
2319 3406    2    8 

In [5]:
table(da$DMDCITZN)


   1    2    7    9 
4746  975    8    5 

In [6]:
# Converting numerical column into character
da$RIAGENDR <- as.character(da$RIAGENDR)

# 1-> Male and 2->Female
da <- da %>% 
    mutate(
    RIAGENDR = ifelse(RIAGENDR == 1, "Male", ifelse(RIAGENDR == 2, "Female", RIAGENDR)),
    SMQ020 = ifelse(SMQ020 == 1, "Yes", ifelse(SMQ020 == 2, "No", 
                                               ifelse(SMQ020 %in% c(7,9), NA, SMQ020))),
    DMDCITZN = ifelse(DMDCITZN == 1, "Yes", ifelse(DMDCITZN == 2, "No",
                                                   ifelse(DMDCITZN %in% c(7,9), NA, DMDCITZN))))

In [7]:
head(da)

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,⋯,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,83732,1,NA,1,Yes,Male,62,3,Yes,5,⋯,124,64,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2
2,83733,1,NA,6,Yes,Male,53,3,No,3,⋯,140,88,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NA
3,83734,1,NA,NA,Yes,Male,78,3,Yes,3,⋯,132,44,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2
4,83735,2,1,1,No,Female,56,3,Yes,5,⋯,134,68,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2
5,83736,2,1,1,No,Female,42,4,Yes,4,⋯,114,54,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2
6,83737,2,2,NA,No,Female,72,1,No,2,⋯,122,58,64.4,150.0,28.6,34.4,33.5,31.4,92.9,NA


## 1) Hypothesis Tests for One Proportion

The most basic hypothesis test may be the **one-sample test for a proportion**. This test is used if we have specified a particular value as the null value for the proportion, and we wish to assess if the data are compatible with the true parameter value being equal to this specified value. One-sample tests are not used very often in practice, because it is not very common that we have a specific fixed value to use for comparison.

For illustration, imagine that the rate of lifetime smoking in another country was known to be 40%, and we wished to assess whether the rate of lifetime smoking in the US were different from 40%. In the following notebook cell, we carry out the **(two-sided) one-sample test** that the population proportion of smokers is 0.4, and obtain a p-value of 0.43. This indicates that the NHANES data are compatible with the proportion of (ever) smokers in the US being 40%.

In [8]:
x <- da[da[!is.na(da$SMQ020),]$SMQ020 == "Yes",]

### Method 1

In [9]:
# Estimate of the proportion
phat <- 0.4 

# Filter out the NA's from the SMQ020 column of smokers:
x <- na.omit(da$SMQ020)== "Yes"

# proportion of smokers (sample)
 p <- mean(x)

#Standard Error of proportion:
se <- sqrt(phat * (1-phat) / length(x))

#Test statistic and p-value
test_stat <- (p - phat ) / se
pvalue <- 2 * pnorm(-abs(test_stat))

cat("Test statistic =", test_stat, "and the p-value=", pvalue)

Test statistic = 0.7823564 and the p-value= 0.4340052

### Method 2

In [10]:
# Two-sided Z-test for proportions
n <- length(x)
z_test_result <- prop.test(x= sum(x), n = n, p = phat, alternative = "two.sided", correct = FALSE)

# Extract p-value from the test result
z_test_stat <- z_test_result$statistic
p_value_z_test <- z_test_result$p.value

cat("Test statistic =",z_test_stat, "Two-sided Z-test p-value:", p_value_z_test, "\n")

Test statistic = 0.6120815 Two-sided Z-test p-value: 0.4340052 


In [11]:
# Binomial test
binom_test_result <- binom.test(sum(x), n, p = phat)

# Extract p-value from the binomial test result
p_value_binom_test <- binom_test_result$p.value

cat("Binomial test p-value:", p_value_binom_test, "\n")

Binomial test p-value: 0.4340361 


## 2) Hypothesis Tests for Two Proportions

Comparative tests tend to be used much more frequently than tests comparing one population to a fixed value. A **two-sample test of proportions** is used to assess whether the proportion of individuals with some trait differs between two sub-populations. For example, we can compare the smoking rates between females and males. Since smoking rates vary strongly with age, we do this in the subpopulation of people between 20 and 25 years of age. In the cell below, we carry out this test without using any libraries. We find that the smoking rate for men is around 10 percentage points greater than the smoking rate for females, and this difference is statistically significant (the p-value is around 0.01).

### Method 1

In [12]:
dx <- da[c("SMQ020", "RIAGENDR", "RIDAGEYR")]
dx <- dx[complete.cases(dx),]

# Proportion and number of smokers per gender
p <- dx %>%
  group_by(RIAGENDR) %>%
  summarise(
    Smoke = mean(SMQ020 == "Yes"),
    N = n()
  )

# Rename columns, and rows
colnames(p) <- c("RIAGENDR", "Smoke", "N")
rownames(p) <- p$RIAGENDR

# Print the result
print(p)

Warning message:
"Setting row names on a tibble is deprecated."


# A tibble: 2 × 3
  RIAGENDR Smoke     N
* <chr>    <dbl> <int>
1 Female   0.305  2972
2 Male     0.513  2753


Essentially the same test as above can be conducted by converting the "Yes"/"No" responses to numbers (Yes=1, No=0) and conducting a two-sample t-test, as below:

In [13]:
# combined proportion
p_comb <- mean(dx$SMQ020 == "Yes")
va <- p_comb * (1 - p_comb)

# Standard Error
se <- sqrt(va * (1/ 1/p["Female","N"]+ 1/p["Male","N"]))

In [14]:
#Test-statistic
test_stat <- unlist((p["Female","Smoke"] - p["Male","Smoke"] )/ se)

#p-value
pvalue <- 2 * pnorm(-abs(test_stat))

cat("test-statistic=", test_stat, "pvalue=", pvalue)

test-statistic= -16.04972 pvalue= 5.742289e-58

### Method 2 

In [15]:
colnames(dx)

[1] "SMQ020"   "RIAGENDR" "RIDAGEYR"

In [16]:
dx_females <- dx[dx$RIAGENDR == "Female",] %>% 
    mutate(
        SMQ020 = ifelse(SMQ020 == "Yes", 1, 0)
    )

dx_male <- dx[dx$RIAGENDR == "Male",] %>% 
    mutate(
    SMQ020 = ifelse(SMQ020 == "Yes", 1, 0))

In [17]:
# test

t_test_result <- t.test(x = dx_females$SMQ020, y = dx_male$SMQ020)

In [18]:
t_test_result$statistic

t 
-16.36908

In [19]:
t_test_result$p.value

[1] 7.339854e-59

# 3) Hypothesis Tests Comparing Means

Tests of means are similar in many ways to tests of proportions. Just as with proportions, for comparing means there are **one** and **two-sample tests**, **z-tests** and **t-tests**, and **one-sided** and **two-sided tests**. As with tests of proportions, one-sample tests of means are not very common, but we illustrate a one sample test in the cell below. We compare systolic blood pressure `BPXSY1` to the fixed value 120 (which is the lower threshold for "pre-hypertension"), and find that the mean is significantly different from 120 (the point estimate of the mean is 126).

### One-Sample Test

In [20]:
colnames(da)

[1] "SEQN"     "ALQ101"   "ALQ110"   "ALQ130"   "SMQ020"   "RIAGENDR"
 [7] "RIDAGEYR" "RIDRETH1" "DMDCITZN" "DMDEDUC2" "DMDMARTL" "DMDHHSIZ"
[13] "WTINT2YR" "SDMVPSU"  "SDMVSTRA" "INDFMPIR" "BPXSY1"   "BPXDI1"  
[19] "BPXSY2"   "BPXDI2"   "BMXWT"    "BMXHT"    "BMXBMI"   "BMXLEG"  
[25] "BMXARML"  "BMXARMC"  "BMXWAIST" "HIQ210"

In [21]:
dx <- da[,c("BPXSY1", "RIDAGEYR", "RIAGENDR")]
dx <- dx[complete.cases(dx),]
head(dx)

,BPXSY1,RIDAGEYR,RIAGENDR
,<dbl>,<int>,<chr>
1,128,62,Male
2,146,53,Male
3,138,78,Male
4,132,56,Female
5,100,42,Female
6,116,72,Female


In [22]:
# Male, age between 40 and 50:
dx <- dx  %>% 
    filter((RIAGENDR == "Male") & (RIDAGEYR >=40) & (RIDAGEYR <=50))

In [23]:
mean(dx$BPXSY1)

[1] 125.867

In [24]:
#test statistic and p-value :
library(BSDA)

z.test(x = dx$BPXSY1, mu = 120, sigma.x = sd(dx$BPXSY1))


	One-sample z-Test

data:  dx$BPXSY1
z = 7.4698, p-value = 8.034e-14
alternative hypothesis: true mean is not equal to 120
95 percent confidence interval:
 124.3276 127.4064
sample estimates:
mean of x 
  125.867 


### Two-Sample Test

In the cell below, we carry out a formal test of the null hypothesis that the mean blood pressure for women between the ages of 50 and 60 is equal to the mean blood pressure of men between the ages of 50 and 60. The results indicate that while the mean systolic blood pressure for men is slightly greater than that for women (129 mm/Hg versus 128 mm/Hg), this difference is not statistically significant.

There are a number of different variants on the two-sample t-test. Two often-encountered variants are the t-test carried out using the t-distribution, and the t-test carried out using the normal approximation to the reference distribution of the test statistic, often called a z-test. Below we display results from both these testing approaches. When the sample size is large, the difference between the t-test and z-test is very small.

In [25]:
dx <- na.omit(da[c("BPXSY1", "RIDAGEYR", "RIAGENDR")])
dx <- dx %>% 
    filter((dx$RIDAGEYR>=50) & (dx$RIDAGEYR<=60))
head(dx)

,BPXSY1,RIDAGEYR,RIAGENDR
,<dbl>,<int>,<chr>
1,146,53,Male
2,132,56,Female
3,178,56,Male
4,134,57,Female
5,136,54,Female
6,116,58,Female


In [26]:
bpx_female <- dx[dx$RIAGENDR == "Female", "BPXSY1"]
bpx_male <- dx[dx$RIAGENDR == "Male", "BPXSY1"]

cat(mean(bpx_female), " ",mean(bpx_male))

127.9256   129.2383

In [27]:
# Z-test :
library(BSDA)

z.test(x = bpx_female, y = bpx_male, sigma.x =  sd(bpx_female), sigma.y = sd(bpx_male))


	Two-sample z-Test

data:  bpx_female and bpx_male
z = -1.1055, p-value = 0.2689
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -3.639892  1.014536
sample estimates:
mean of x mean of y 
 127.9256  129.2383 


In [28]:
# T-test:
t.test(x = bpx_female, y = bpx_male)


	Welch Two Sample t-test

data:  bpx_female and bpx_male
t = -1.1055, df = 951.47, p-value = 0.2692
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -3.642856  1.017500
sample estimates:
mean of x mean of y 
 127.9256  129.2383 


Another important aspect of two-sample mean testing is **"heteroscedasticity"**, meaning that the variances within the two groups being compared may be different. While the goal of the test is to compare the means, the variances play an important role in calibrating the statistics (deciding how big the mean difference needs to be to be declared statistically significant). In the NHANES data, we see that there are moderate differences between the amount of variation in BMI for females and for males, looking within 10-year age bands. In every age band, females having greater variation than males.

In [34]:
# Subset the data and drop NA values
dx <- na.omit(select(da, BPXSY1, RIDAGEYR, RIAGENDR))

# Age grouping
da <- mutate(da, agegrp = cut(RIDAGEYR, breaks = c(18, 30, 40, 50, 60, 70, 80)))

# Calculate standard deviation by age group and gender
result <- da %>%
  group_by(agegrp, RIAGENDR) %>%
  summarise(std_BMXBMI = sd(BMXBMI, na.rm =  TRUE), .groups = "drop") %>%
  pivot_wider(names_from = RIAGENDR, values_from = std_BMXBMI)

# Print the result
print(result)

# A tibble: 7 × 3
  agegrp  Female  Male
  <fct>    <dbl> <dbl>
1 (18,30]   7.75  6.65
2 (30,40]   8.32  6.62
3 (40,50]   8.08  6.41
4 (50,60]   7.58  5.91
5 (60,70]   7.60  5.93
6 (70,80]   6.28  4.97
7 NA        7.18  7.48


The standard error of the mean difference (e.g. mean female blood pressure minus mean male blood pressure) can be estimated in at least two different ways. In the statsmodels library, these approaches are referred to as the **"pooled"** and the **"unequal" approach to estimating the variance**. If the variances are equal (i.e. there is no heteroscedasticity), then there should be little difference between the two approaches. Even in the presence of moderate heteroscedasticity, as we have here, we can see that the results for the two methods are quite similar. Below we have a loop that considers each 10-year age band and assesses the evidence for a difference in mean BMI for women and for men. The results printed in each row of output are the test-statistic and p-value.

In [40]:
# Function to perform Z-test for two independent samples
perform_ztest <- function(sub_data, group_var) {
  bmi_female <- subset(sub_data, RIAGENDR == "Female")$BMXBMI
  bmi_male <- subset(sub_data, RIAGENDR == "Male")$BMXBMI
  
 # Drop NA values
  bmi_female <- bmi_female[complete.cases(bmi_female)]
  bmi_male <- bmi_male[complete.cases(bmi_male)]
  
  # Perform Z-test
  result_pooled <- t.test(bmi_female, bmi_male, var.equal = TRUE)
  result_unequal <- t.test(bmi_female, bmi_male, var.equal = FALSE)
  
  # Print results
  cat(group_var, "\n")
  cat("pooled: t =", result_pooled$statistic, ", p =", result_pooled$p.value, "\n")
  cat("unequal: t =", result_unequal$statistic, ", p =", result_unequal$p.value, "\n\n")
}



# Age grouping
da <- mutate(da, agegrp = cut(RIDAGEYR, breaks = c(18, 30, 40, 50, 60, 70, 80)))

# Perform Z-tests for each age group
da %>%
  group_by(agegrp) %>%
  do(perform_ztest(., group_var = unique(.$agegrp)))

1 
pooled: t = 1.702693 , p = 0.08889832 
unequal: t = 1.717461 , p = 0.08616718 

2 
pooled: t = 1.437828 , p = 0.1508233 
unequal: t = 1.443787 , p = 0.1491517 

3 
pooled: t = 2.893376 , p = 0.003903827 
unequal: t = 2.967869 , p = 0.003078687 

4 
pooled: t = 3.362109 , p = 0.0008055543 
unequal: t = 3.375494 , p = 0.0007692272 

5 
pooled: t = 3.61724 , p = 0.000315093 
unequal: t = 3.628483 , p = 0.0003027012 

6 
pooled: t = 2.926729 , p = 0.003523629 
unequal: t = 2.93778 , p = 0.003405544 

NA 
pooled: t = 0.04742079 , p = 0.9622512 
unequal: t = 0.04734629 , p = 0.9623115 



ERROR: [1m[33mError[39m in `do()`:[22m
[1m[22m[33m![39m Results 1, 2, 3, 4, 5, ... must be data frames, not NULL.
